In [132]:
import pandas as pd
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas
from geopy.geocoders import Nominatim

## Geotagging the scraped Data

In [133]:
df = pd.read_csv('all_reviews.csv')
locations_df = pd.read_csv('locations_geotagged.csv')
beer_df = pd.read_csv('beer_deets.csv')

In [134]:
df

,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit
...,...,...,...,...,...,...
31142,Perlenbacher Lager,3.7,+1.4,Lidl Europe,Nice beer. Even nicer when it's really cold. B...,BarleyScout55
31143,Northern Monk Patrons Project 14.05 King - Gra...,1.4,-1.8,Northern Monk Brew Co.,"440ml can at We Brought Beer, Tooting. Murky ...",Beese
31144,North High Grapefruit Walleye,1.5,-1.8,North High Brewing,$2 orange can as pictured. First thought? Who ...,cheap
31145,Harboe Dansk Julebryg,5.0,+2.8,Harboes Bryggeri,Rigtig dejlig juleøl til billige penge! En sik...,FrederikThordal


In [135]:
locations_df

,locations,latitude,longitude,country
0,"Bungay, Suffolk",52.456225,1.437341,United Kingdom
1,"Faversham, Kent",51.314409,0.891189,United Kingdom
2,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom
3,"Round Rock, Texas",30.508235,-97.678893,United States of America
4,"Brooklyn, New York",40.650104,-73.949582,United States of America
...,...,...,...,...
2481,"Rotherham, South Yorkshire",53.431042,-1.355187,United Kingdom
2482,"Penn Yann, New York",NaN,NaN,NaN
2483,"Tortworth, Gloucestershire",51.636745,-2.431266,United Kingdom
2484,"Merzig, Saarland",49.442702,6.637490,Germany


In [136]:
geolocator = Nominatim(user_agent="None")

In [137]:
locations_new = pd.DataFrame(df.brewery_location[~df.brewery_location.isin(locations_df.locations)].unique(),columns=['locations'])
print(len(locations_new), "new locations to search for! Will take", len(locations_new)/2, "seconds to run.")

0 new locations to search for! Will take 0.0 seconds to run.


In [138]:
def tryconvert(x):
        try:
            return geolocator.geocode(x).latitude
        except:
            return None
    

In [139]:
def tryconvertlong(x):
        try:
            return geolocator.geocode(x).longitude
        except:
            return None

In [140]:
locations_new["latitude"] = locations_new.locations.apply(lambda x: tryconvert(x))

In [141]:
locations_new["longitude"] = locations_new.locations.apply(lambda x: tryconvertlong(x))

In [142]:
countries = []
for row in locations_new.index:
    try:
        loc = geolocator.reverse([locations_new.iloc[row,1],locations_new.iloc[row,2]], language='en')
        countries.append(loc.raw['address']['country'])
    except:
        countries.append(None)
        
    

In [143]:
locations_new['country'] = countries

In [144]:
locations_new

,locations,latitude,longitude,country


In [145]:
locations_df = pd.concat([locations_df,locations_new])
locations_df.to_csv('locations_geotagged.csv',index=False)

## Merging and cleaning

In [146]:
beer_df

,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...
16687,Prairie Skonbeit\n🇺🇸 Stout - Imperial Flavored...,🇺🇸,Stout - Imperial Flavored,NaN,-,-,4.00,1.0,Prairie Skonbeit
16688,Prairie Moken Troll\n🇺🇸 Stout - Imperial Flavo...,🇺🇸,Stout - Imperial Flavored,NaN,-,-,3.60,1.0,Prairie Moken Troll
16689,Hübner Bräu Wattendorf 3-Ährenbier\n🇩🇪 Special...,🇩🇪,Specialty Grain - Other,0.0,-,-,3.10,5.0,Hübner Bräu Wattendorf 3-Ährenbier
16690,Hübner Bräu Wattendorf Naturtrübes Zwickelpils...,🇩🇪,Zwickelbier / Kellerbier / Landbier,0.0,52,90,3.40,10.0,Hübner Bräu Wattendorf Naturtrübes Zwickelpils


In [147]:
# deal with the price issue
price_df = beer_df[beer_df.rb_overall_score.str.contains('Available')].copy()
beer_df = beer_df[~beer_df.rb_overall_score.str.contains('Available')]
price_df['rb_overall_score'] = price_df.return_string.apply(lambda x: x.split('\n')[3])
price_df['rb_style_score'] = price_df.return_string.apply(lambda x: x.split('\n')[4])
price_df['rb_user_rating'] = price_df.return_string.apply(lambda x: x.split('\n')[5])
price_df['rb_rating_number'] = price_df.return_string.apply(lambda x: x.split('\n')[6].replace("(","").replace(")",""))
beer_df = pd.concat([beer_df,price_df])
print("Number of issue beers sorted:", len(price_df))


Number of issue beers sorted: 290


In [148]:
# set the types to float (cant do for the overall score and rating as there are two many None "-")
beer_df.rb_user_rating = beer_df.rb_user_rating.astype(float)
beer_df.rb_rating_number = beer_df.rb_rating_number.astype(float)

In [149]:
# Creating DF with all details
df_all = pd.merge(df,locations_df,how='left',left_on='brewery_location',right_on='locations')
df_all = pd.merge(df_all, beer_df, how='left', left_on='beer_name',right_on='name_found')
df_all


,beer_name,beer_rating,brewery_location,brewery_name,review_content,reviewer_name,locations,latitude,longitude,country,return_string,flag,type,abv,rb_overall_score,rb_style_score,rb_user_rating,rb_rating_number,name_found
0,St. Peter's Ruby Red Ale,3.3,"Bungay, Suffolk",St. Peter's Brewery,"Ruby color, offwhite coam. Aroma and taste: fl...",alex_leit,"Bungay, Suffolk",52.456225,1.437341,United Kingdom,St. Peter's Ruby Red Ale\n🇬🇧 Bitter - Ordinary...,🇬🇧,Bitter - Ordinary / Best,4.3,49,95,3.25,415.0,St. Peter's Ruby Red Ale
1,Shepherd Neame Spitfire (Bottle),2.2,"Faversham, Kent",Shepherd Neame,Disappointing show from the lads at ‘Britain’s...,shivermetimber.gray,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Spitfire (Bottle)\n🇬🇧 Bitter - ...,🇬🇧,Bitter - Ordinary / Best,4.5,34,59,3.05,1083.0,Shepherd Neame Spitfire (Bottle)
2,Arbor Rocketman,3.8,"Bristol, Gloucestershire",Arbor Ales,"Puszka. Mętne o żółtym kolorze. Piana biała,ob...",MarcinG,"Bristol, Gloucestershire",51.852767,-2.255369,United Kingdom,Arbor Rocketman\n🇬🇧 IPA • 6.0%\n95\n97\n3.74\n...,🇬🇧,IPA,6.0,95,97,3.74,59.0,Arbor Rocketman
3,Shepherd Neame India Pale Ale (Bottle),3.4,"Faversham, Kent",Shepherd Neame,Bottle. Amber with bronze color and white foam...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame India Pale Ale (Bottle)\n🇬🇧 IPA...,🇬🇧,IPA - English,6.1,46,47,3.22,420.0,Shepherd Neame India Pale Ale (Bottle)
4,Shepherd Neame Christmas Ale,3.3,"Faversham, Kent",Shepherd Neame,Bottle. Aroma: English hops and malts with tof...,alex_leit,"Faversham, Kent",51.314409,0.891189,United Kingdom,Shepherd Neame Christmas Ale\n🇬🇧 Strong Ale - ...,🇬🇧,Strong Ale - English,7.0,45,46,3.18,409.0,Shepherd Neame Christmas Ale
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31431,Perlenbacher Lager,3.7,+1.4,Lidl Europe,Nice beer. Even nicer when it's really cold. B...,BarleyScout55,+1.4,48.780357,10.056559,Germany,Perlenbacher Lager\n🇩🇪 Pale Lager - North Am. ...,🇩🇪,Pale Lager - North Am. / Light / Rice,5.0,5,66,2.38,33.0,Perlenbacher Lager
31432,Northern Monk Patrons Project 14.05 King - Gra...,1.4,-1.8,Northern Monk Brew Co.,"440ml can at We Brought Beer, Tooting. Murky ...",Beese,-1.8,48.780981,10.037026,Germany,Northern Monk Patrons Project 14.05 King - Gra...,🇬🇧,IPA - Flavored,7.4,48,22,3.28,18.0,Northern Monk Patrons Project 14.05 King - Gra...
31433,North High Grapefruit Walleye,1.5,-1.8,North High Brewing,$2 orange can as pictured. First thought? Who ...,cheap,-1.8,48.780981,10.037026,Germany,North High Grapefruit Walleye\n🇺🇸 IPA - Flavor...,🇺🇸,IPA - Flavored,4.5,53,45,3.21,19.0,North High Grapefruit Walleye
31434,Harboe Dansk Julebryg,5.0,+2.8,Harboes Bryggeri,Rigtig dejlig juleøl til billige penge! En sik...,FrederikThordal,+2.8,54.379170,10.982190,Germany,Harboe Dansk Julebryg\n🇩🇰 Amber Lager - Intl /...,🇩🇰,Amber Lager - Intl / Vienna,5.7,4,6,2.19,46.0,Harboe Dansk Julebryg


In [150]:
# drop excess columns
df_all.drop(["locations","return_string","name_found"],axis=1,inplace=True)
# drop rows with ratings instead of location
print(len(df_all[df_all['brewery_location'].str.len()<4]),"columns without locations dropped.")
df_all = df_all[~df_all['brewery_location'].str.len()<4]
# drop all NaNs
print(len(df_all) - len(df_all.dropna()), "columns with NaNs dropped.")
df_all.dropna(inplace=True)


220 columns without locations dropped.
2359 columns with NaNs dropped.


In [151]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 29077 entries, 0 to 31435
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   beer_name         29077 non-null  object 
 1   beer_rating       29077 non-null  float64
 2   brewery_location  29077 non-null  object 
 3   brewery_name      29077 non-null  object 
 4   review_content    29077 non-null  object 
 5   reviewer_name     29077 non-null  object 
 6   latitude          29077 non-null  float64
 7   longitude         29077 non-null  float64
 8   country           29077 non-null  object 
 9   flag              29077 non-null  object 
 10  type              29077 non-null  object 
 11  abv               29077 non-null  float64
 12  rb_overall_score  29077 non-null  object 
 13  rb_style_score    29077 non-null  object 
 14  rb_user_rating    29077 non-null  float64
 15  rb_rating_number  29077 non-null  float64
dtypes: float64(6), object(10)
memory usage: 

## Deal with Non-English Descriptions

## Save down cleaned data

In [152]:
df_all.to_csv("data_clean.csv",index=False)